## Expose our ML models to front-end consumers using Flask-RestPlus

TODO: Go into more detail about Flask and how the code below works.

In [ ]:
import os
import pickle
from flask import Flask
from flask_restplus import Api, Resource, fields, reqparse
from werkzeug.contrib.fixers import ProxyFix
import pandas as pd
import numpy as np

app = Flask(__name__)
app.wsgi_app = ProxyFix(app.wsgi_app)
app.config.SWAGGER_UI_DOC_EXPANSION = 'full'
api = Api(app, version='1.0', title='House Prediction API',
    description='Expose ML Models as endpoints',
)

ns = api.namespace('api/v1', description='House prediction API')

AVAILABLE_MODELS = ['xgb', 'mlp', 'mlr', 'rf']
AVAILABLE_PROPERTY_TYPES = []

def load_models() -> dict:
    with open('./models/sf/multiple_linear.pkl', 'rb') as f:
        linear_model = pickle.load(f)
    with open('./models/sf/gbm.pkl', 'rb') as f:
        gbm_model = pickle.load(f)
    with open('./models/sf/xgb.pkl', 'rb') as f:
        xgb_model = pickle.load(f)
    with open('./models/sf/random_forest.pkl', 'rb') as f:
        rf_model = pickle.load(f)
    with open('./models/sf/mlp.pkl', 'rb') as f:
        mlp_model = pickle.load(f)
    models = {
        'xgb': xgb_model,
             'mlp': mlp_model,
             'mlr': linear_model,
             'rf': rf_model
    }
    return models

MODELS = load_models()

def ensemble_prediction(input_data: pd.DataFrame) -> dict:
    predictions = {}
    avg = []
    for model_type in AVAILABLE_MODELS:
        prediction = MODELS[f'{model_type}'].predict(input_data)
        prediction = float(prediction.squeeze())
        predictions[model_type] = prediction
        avg.append(prediction)
    predictions['ensemble'] = np.average(np.array(avg))
    return predictions

def predict(input_dict: dict, model_type: str) -> float:
    df_cols = pd.read_csv('./data/sf/data_clean_engineered.csv')
    features = [feature for feature in df_cols.columns if feature != 'price']
    df_input = pd.get_dummies(pd.DataFrame(data=[input_dict], columns=features).fillna(0))
    
    if 'ensemble' in model_type.lower():
        return ensemble_prediction(df_input)
    elif model_type.lower() not in AVAILABLE_MODELS:
        raise Exception(f"model type {model_type} not available. Available models: {AVAILABLE_MODELS} or ensemble")

    prediction = MODELS[f'{model_type}'].predict(df_input)
    return float(prediction.squeeze())

@ns.route('/prediction')
class Prediction(Resource):
    '''Prediction Endpoint'''
    @ns.param('bed', f'Number of bedrooms',
          type=int,
             required=True)
    @ns.param('bath', f'Number of bathrooms',
          type=int,
             required=True)
    @ns.param('sqft', f'Square footage',
          type=int,
             required=True)
    @ns.param('zipcode', f'Zip code',
          type=str,
             required=True)
    @ns.param('property_type', f'Type of property (choices: {AVAILABLE_PROPERTY_TYPES})',
          type=str,
             required=True)
    @ns.param('model', f'Type of ML model to use (choices: {AVAILABLE_MODELS})',
          type=str,
             required=True,
             default='ensemble')
    def get(self):
        '''Get prediction'''
        try:
            parser = reqparse.RequestParser()
            parser.add_argument('bed', type=int, required=True)
            parser.add_argument('bath', type=int, required=True)
            parser.add_argument('sqft', type=int, required=True)
            parser.add_argument('zipcode', type=int, required=True)
            parser.add_argument('property_type', type=str, required=True)
            parser.add_argument('model', type=str, required=True)
            args = parser.parse_args()
            bed = args['bed']
            bath= args['bath']
            sqft= args['sqft']
            zipcode= args['zipcode']
            property_type= args['property_type']
            model = args['model']
            input_dict = {
                'bed': bed,
                'bath': bath,
                'sqft': sqft,
                'postal_code_{}'.format(zipcode): 1,
                'property_type_{}'.format(property_type): 1,
                         }
            prediction = predict(input_dict=input_dict, model_type=model)

            return {'prediction': prediction}, 200
        except Exception as e:
            return {'error': str(e)}, 500

if __name__ == '__main__':
    load_models()
    app.run(host="0.0.0.0", port=int(os.environ['API_PORT']))

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.21.0.1 - - [10/Mar/2018 01:41:18] "GET /api/v1/prediction?model=ensemble&property_type=condo&zipcode=94109&sqft=825&bath=1&bed=1 HTTP/1.1" 200 -
172.21.0.1 - - [10/Mar/2018 01:41:20] "GET /api/v1/prediction?model=ensemble&property_type=condo&zipcode=94109&sqft=825&bath=1&bed=1 HTTP/1.1" 200 -
172.21.0.1 - - [10/Mar/2018 01:43:06] "GET /api/v1/prediction?model=ensemble&property_type=condo&zipcode=94105&sqft=1223&bath=2&bed=2 HTTP/1.1" 200 -
